In [1]:
import pyomo.environ as pyo
import numpy as np

In [2]:
M = 1000
e = np.e

In [3]:
# Инициализация модели
model = pyo.ConcreteModel(name = 'Optimal decomposition')

In [4]:
# Ограничения модели
model.constraints = pyo.ConstraintList()

In [5]:
model.f = pyo.VarList(domain=pyo.Binary)

In [6]:
model.a = pyo.Var(domain=pyo.Binary)

In [ ]:
# lambda x: 0 if x <= 0 else 1
def check_include(vector):
    result = []
    for x in vector:
        f = model.f.add()
        result.append(f)
        model.constraints.add((x - M * f <= 0))
        model.constraints.add((M * f - x >= 0))
    return np.array(result)

In [7]:
# lambda x: 0 if x < 1 else 1
def check_implemented(x):
    result = []
    for el in x:
        f = model.f.add()
        result.append(f)
        model.constraints.add((f - M * (el - 1) >= 0))
        model.constraints.add((f - el <= 0))
    return np.array(result)

In [8]:
def solve(solver_name):
    solver = pyo.SolverFactory(solver_name)
    instance = model.create_instance()
    result = solver.solve(instance)
    instance.display()

In [9]:
model.OBJ = pyo.Objective(expr = sum( check_implemented([model.a]) ), sense=pyo.minimize)

In [10]:
solve('glpk')

Model Optimal decomposition

  Variables:
    f : Size=1, Index={1}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :     1 : False : False : Binary
    a : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   0.0

  Constraints:
    constraints : Size=2
        Key : Lower : Body   : Upper
          1 :   0.0 : 1000.0 :  None
          2 :  None :    0.0 :   0.0


In [260]:
def test_im(x, f):
    res1 = x - f - M * (1 - f) <= 0
    res2 = -x + M * f >= 0
    return(res1 and res2)

In [261]:
test_im(-0.5, 0) # True

True

In [262]:
test_im(-0.5, 1) # False

True

In [263]:
test_im(0, 0) # True

True

In [264]:
test_im(0, 1) # False

True

In [265]:
test_im(0.5, 1) # True

True

In [266]:
test_im(0.5, 0) # False

False

In [267]:
test_im(1, 1) # True

True

In [268]:
test_im(1, 0) # False

False

In [269]:
test_im(1.5, 1) # True

False

In [270]:
test_im(1.5, 1) # False

False

In [50]:
def test_in(x, f):
    res1 = -M * f - (1 - x)
    res2 = M * f + (1 - x)
    return(res1, res2)